In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
fileRoot = "D:\\ICLR-CropDisease\\dataset\\"
zipsize =128


In [2]:
path_healthy = "train\\healthy_wheat\\"
path_leaf = "train\\leaf_rust\\"
path_stem = "train\\stem_rust\\"

In [3]:
def blur(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32)
    return cv2.filter2D(image, -1, kernel=kernel)
def preprocess(res):
    zipsize = 256
    res = cv2.resize(res, dsize=(zipsize, zipsize))
    res = blur(res)
    res = res [64:192,64:192]
    return res
def loadData():
    data_img = []
    data_label = []
    for file in os.listdir(fileRoot + path_healthy):
        img = cv2.imread(fileRoot + path_healthy + file)
        res = preprocess(img)
        data_img.append(res)
        data_label.append(2)
        data_img.append(cv2.flip(res, 1))
        data_label.append(2)
        data_img.append(cv2.flip(res, 0))
        data_label.append(2)
        data_img.append(cv2.flip(res, -1))
        data_label.append(2)

    for file in os.listdir(fileRoot + path_leaf):
        img = cv2.imread(fileRoot + path_leaf + file)
        if img is None:
            print(file)
            continue
        res = preprocess(img)
        data_label.append(0)
        data_img.append(res)
        data_img.append(cv2.flip(res, 1))
        data_label.append(0)
        data_img.append(cv2.flip(res, 0))
        data_label.append(0)
        data_img.append(cv2.flip(res, -1))
        data_label.append(0)

    for file in os.listdir(fileRoot + "train\\stem_rust\\"):
        img = cv2.imread(fileRoot + "train\\stem_rust\\" + file)
        res = preprocess(img)
        data_img.append(res)
        data_label.append(1)
        data_img.append(cv2.flip(res, 1))
        data_label.append(1)
        data_img.append(cv2.flip(res, 0))
        data_label.append(1)
        data_img.append(cv2.flip(res, -1))
        data_label.append(1)
        
    for i in range(len(data_img)):
        data_img[i] = data_img[i] / 255
    data_img = np.array(data_img)
    return data_img, data_label

In [4]:
def trainTestSplit(data_img,data_label):

    X_train, X_test, y_train, y_test = train_test_split(data_img,data_label,test_size = 0.3)
    y_train =np.array(y_train)
    y_test =np.array(y_test)
    return X_train, X_test, y_train, y_test

In [5]:
def trainModel(X_train, X_test, y_train, y_test):
   
    with tf.device('/gpu:0'):
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(zipsize, zipsize, 3)))
        tf.keras.layers.Dropout(0.2)
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        tf.keras.layers.Dropout(0.2)
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        tf.keras.layers.Dropout(0.2)
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        tf.keras.layers.Dropout(0.2)
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        tf.keras.layers.Dropout(0.2)
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(3,activation='softmax'))

        model.compile(optimizer='Adamax',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

        # Train
        history = model.fit(X_train, y_train,batch_size = 16, epochs=100,use_multiprocessing = True)

        # test
        #test_loss, test_acc = model.evaluate(X_test,y_test, batch_size = 16, verbose=2)
        #print(test_acc)
        return model

In [6]:
data_img,data_lable = loadData()
print("DATA READY")

7U06EV.gif
DATA READY


In [7]:
#X_train, X_test, y_train, y_test = trainTestSplit(data_img,data_lable)

#model = trainModel(X_train, X_test, y_train, y_test)
data_label = np.array(data_lable)
model = trainModel(data_img,[],data_label,[])

Train on 3500 samples
Epoch 1/100
3500/3500 [==============================] - 4s 1ms/sample - loss: 1.0227 - accuracy: 0.4843
Epoch 2/100
3500/3500 [==============================] - 2s 554us/sample - loss: 0.9546 - accuracy: 0.5831
Epoch 3/100
3500/3500 [==============================] - 2s 562us/sample - loss: 0.9187 - accuracy: 0.6123
Epoch 4/100
3500/3500 [==============================] - 2s 552us/sample - loss: 0.8804 - accuracy: 0.6629
Epoch 5/100
3500/3500 [==============================] - 2s 556us/sample - loss: 0.8645 - accuracy: 0.6711
Epoch 6/100
3500/3500 [==============================] - 2s 568us/sample - loss: 0.8344 - accuracy: 0.7071
Epoch 7/100
3500/3500 [==============================] - 2s 548us/sample - loss: 0.8298 - accuracy: 0.7126
Epoch 8/100
3500/3500 [==============================] - 2s 569us/sample - loss: 0.8041 - accuracy: 0.7451
Epoch 9/100
3500/3500 [==============================] - 2s 557us/sample - loss: 0.8128 - accuracy: 0.7320
Epoch 10/100
3500

3500/3500 [==============================] - 2s 552us/sample - loss: 0.5988 - accuracy: 0.9531
Epoch 77/100
3500/3500 [==============================] - 2s 548us/sample - loss: 0.5969 - accuracy: 0.9546
Epoch 78/100
3500/3500 [==============================] - 2s 556us/sample - loss: 0.5962 - accuracy: 0.9551
Epoch 79/100
3500/3500 [==============================] - 2s 539us/sample - loss: 0.5962 - accuracy: 0.9551
Epoch 80/100
3500/3500 [==============================] - 2s 554us/sample - loss: 0.5995 - accuracy: 0.9523
Epoch 81/100
3500/3500 [==============================] - 2s 570us/sample - loss: 0.6049 - accuracy: 0.9471
Epoch 82/100
3500/3500 [==============================] - 2s 565us/sample - loss: 0.6132 - accuracy: 0.9374
Epoch 83/100
3500/3500 [==============================] - 2s 571us/sample - loss: 0.5989 - accuracy: 0.9531
Epoch 84/100
3500/3500 [==============================] - 2s 558us/sample - loss: 0.5961 - accuracy: 0.9554
Epoch 85/100
3500/3500 [=================

In [8]:
def loadTest():
    test = []
    name = []
    for file in os.listdir(fileRoot + "test\\"):
        img = cv2.imread(fileRoot + "test\\" + file)
        res = preprocess(img)
        test.append(res)
        name.append(file)
    for i in range(len(test)):
        test[i] = test[i] / 255
    test = np.array(test)
    result = model.predict_proba(test)
    return name, result
name,result = loadTest()

In [9]:
import pandas as pd


output = []
for i in range(len(result)):
    output.append(np.append(result[i],name[i][0:6]).tolist())
my_df = pd.DataFrame(output)
# change order of columns of csv file so the name is in the first column
#print (my_df)
my_df = my_df[[3, 0, 1, 2]]
my_df.columns = ['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat']
my_df.to_csv('submission.csv', index=False)